In [1]:
import cv2

In [2]:
cap= cv2.VideoCapture(0)

In [3]:
status ,photo = cap.read() 

In [4]:
import cv2

# Load the face detection model
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascase.xml')

# Verify if the face detection model is loaded successfully
if face_cascade.empty():
    print("Error: Failed to load the face detection model.")
    exit(1)

# Load the party mask image with transparent background
mask = cv2.imread('mask.png', cv2.IMREAD_UNCHANGED)

# Function to remove the background of an image
def remove_background(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, mask = cv2.threshold(gray, 1, 255, cv2.THRESH_BINARY_INV)
    masked_image = cv2.bitwise_and(image, image, mask=mask)
    return masked_image

# Function to add the party mask overlay on faces
def add_mask(image, faces):
    for (x, y, w, h) in faces:
        resized_mask = cv2.resize(mask, (w, h))
        mask_h, mask_w, _ = resized_mask.shape

        # Adjust the mask position based on face coordinates
        y1, y2 = y, y + mask_h
        x1, x2 = x, x + mask_w

        # Ensure the mask is within the image boundaries
        if x2 > image.shape[1] or y2 > image.shape[0]:
            continue

        # Apply the mask on the face region
        mask_alpha = resized_mask[:, :, 3] / 255.0
        mask_rgb = resized_mask[:, :, :3]
        masked_face = image[y1:y2, x1:x2]
        masked_face = cv2.cvtColor(masked_face, cv2.COLOR_BGR2RGB)
        masked_face = cv2.addWeighted(masked_face, 1.0, mask_rgb, mask_alpha, 0.0)
        masked_face = cv2.cvtColor(masked_face, cv2.COLOR_RGB2BGR)
        image[y1:y2, x1:x2] = masked_face

    return image

# Open the video capture
video_capture = cv2.VideoCapture(0)  # Change to 1 or 2 if you have multiple cameras

# Loop over frames from the video stream
while True:
    # Read the next frame from the video stream
    ret, frame = video_capture.read()

    # Verify if the frame was read successfully
    if not ret:
        print("Error: Failed to read the frame from the video stream.")
        break

    # Convert the frame to RGB (OpenCV uses BGR by default)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Detect faces in the frame
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # Remove the background of the frame
    frame_rgb = remove_background(frame_rgb)

    # Add the party mask overlay on the detected faces
    frame_with_mask = add_mask(frame_rgb, faces)

    # Convert the frame back to BGR for displaying
    frame_bgr = cv2.cvtColor(frame_with_mask, cv2.COLOR_RGB2BGR)

    # Display the resulting frame
    cv2.imshow('Party Mask', frame_bgr)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture and close all windows
video_capture.release()
cv2.destroyAllWindows()


Error: Failed to load the face detection model.


error: OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\objdetect\src\cascadedetect.cpp:1689: error: (-215:Assertion failed) !empty() in function 'cv::CascadeClassifier::detectMultiScale'
